In [3]:
from elasticsearch import Elasticsearch


# Create the client instance
client = Elasticsearch(
    hosts=["http://localhost:9200"]
)

print(client.info())


{'name': 'bc1ce9c2c23c', 'cluster_name': 'docker-cluster', 'cluster_uuid': '4yqb7bh8TTCqgX-ZxotjyQ', 'version': {'number': '8.5.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '4ed5ee9afac63de92ec98f404ccbed7d3ba9584e', 'build_date': '2022-12-05T18:22:22.226119656Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [12]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

ModuleNotFoundError: No module named 'sentence_transformers'

In [9]:
client.indices.delete(index="book_index", ignore_unavailable=True)

{'acknowledged': True}

In [10]:
# Define the mapping
mappings = {
    "mappings":{
        "properties": {
            "title_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": "true",
                "similarity": "cosine",
            }
        }
    }
}


# Create the index
client.indices.create(index="book_index", body=mappings)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'book_index'}

In [11]:
# index data
import json
from urllib.request import urlopen

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/data.json"
response = urlopen(url)
books = json.loads(response.read())

operations = []
for book in books:
    operations.append({"index": {"_index": "book_index"}})
    # Transforming the title into an embedding using the model
    book["title_vector"] = model.encode(book["title"]).tolist()
    operations.append(book)
# client.bulk(index="book_index", operations=operations, refresh=True)

NameError: name 'model' is not defined

In [13]:
books

[{'title': 'The Pragmatic Programmer: Your Journey to Mastery',
  'authors': ['andrew hunt', 'david thomas'],
  'summary': 'A guide to pragmatic programming for software engineers and developers',
  'publish_date': '2019-10-29',
  'num_reviews': 30,
  'publisher': 'addison-wesley'},
 {'title': 'Python Crash Course',
  'authors': ['eric matthes'],
  'summary': 'A fast-paced, no-nonsense guide to programming in Python',
  'publish_date': '2019-05-03',
  'num_reviews': 42,
  'publisher': 'no starch press'},
 {'title': 'Artificial Intelligence: A Modern Approach',
  'authors': ['stuart russell', 'peter norvig'],
  'summary': 'Comprehensive introduction to the theory and practice of artificial intelligence',
  'publish_date': '2020-04-06',
  'num_reviews': 39,
  'publisher': 'pearson'},
 {'title': 'Clean Code: A Handbook of Agile Software Craftsmanship',
  'authors': ['robert c. martin'],
  'summary': 'A guide to writing code that is easy to read, understand and maintain',
  'publish_date':